In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import decomposition

In [2]:
# Storing the file path in 'loc' variable
loc = (r"C:\Users\Minet\Documents\Repositories\Dev10_Final_Project\Datasets\MLModelDatasetPowerBI.csv")
# Importing the file from the file path contained in 'loc' into 'df' 
df = pd.read_csv(loc)

In [3]:
df = df[['GraduationPercentageRate','HouseholdIncome','FundingPerStudent', 'DropoutPercentageRate']]
df.head()

,GraduationPercentageRate,HouseholdIncome,FundingPerStudent,DropoutPercentageRate
0,90,59328.0,23833.02,3
1,92,63986.4,14013.61,5
2,83,54818.3,24128.02,10
3,95,76903.8,17552.56,3
4,71,61049.2,22382.65,12


In [4]:
# nysd = datasets.load_nystatedistricts()
features=['HouseholdIncome','FundingPerStudent', 'DropoutPercentageRate']
# features=['HouseholdIncome']
target=['GraduationPercentageRate']

X = df[features]
print(X.shape)

y = df.iloc[:, 0]
print(y.shape)

(610, 3)
(610,)


In [5]:
# Create training and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
std_feats = scaler.fit(X_train)

# X_scaled = scaler.fit_transform(X)
# X_scaled = pd.DataFrame(df = X_scaled, columns = X.columns)

# reduce the dimension of features by creating new features which have most of the varience of the original data
pca = decomposition.PCA()

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Create an instance of Lasso Regression implementation
lasso = Lasso(alpha=0.0, normalize=True)

(457, 3)
(153, 3)
(457,)
(153,)


In [6]:
# helps us by passing modules one by one through GridSearchCV for which we want to get the best parameters.
pipe = Pipeline(steps=[("scaler", scaler),
                           ("pca", pca),
                           ("lasso", lasso)])

# for GridSearchCV

# for pca, n_components signifies the number of components to keep after reducing the dimension.
n_components = list(range(1,X.shape[1]+1,1))

# Logistic Regression requires two parameters "normalize" and "selection" to be optimised by GridSearchCV.
normalize = [True, False]
selection = ["cyclic", "random"]
alpha = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
fit_intercept = [True, False]

# creating a dictionary to set all the parameters options for different modules.
parameters = dict(pca__n_components=n_components,
                      lasso__normalize=normalize,
                      lasso__selection=selection,
                      lasso__alpha=alpha,
                      lasso__fit_intercept=fit_intercept)

# Making an object clf_GS for GridSearchCV and fitting the X and y
clf = GridSearchCV(pipe, parameters)
clf.fit(X, y)

# print the results. It will give the values of hyperparameters as a result.
print("Best Number Of Components:", clf.best_estimator_.get_params()["pca__n_components"])
print(clf.best_estimator_.get_params()["lasso"])

CV_Result = cross_val_score(clf, X, y, cv=10, n_jobs=-1, scoring="r2")
print(CV_Result)
print(CV_Result.mean())
print(CV_Result.std())

E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\pipeline.py:382: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.p

E:\Anaconda\lib\site-packages\sklearn\pipeline.py:382: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.757e+03, tolerance: 3.278e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\li

E:\Anaconda\lib\site-packages\sklearn\pipeline.py:382: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.291e+03, tolerance: 3.442e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\li

E:\Anaconda\lib\site-packages\sklearn\pipeline.py:382: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.952e+06, tolerance: 3.926e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\li

E:\Anaconda\lib\site-packages\sklearn\pipeline.py:382: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.940e+06, tolerance: 3.904e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\li

...

E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to 

E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline impor

E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline impor

E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to 

Best Number Of Components: 3
Lasso(alpha=0.0, normalize=True)
[0.6459853  0.09316907 0.8208703  0.68848555 0.81277301 0.6486641
 0.70782531 0.69431586 0.48825047 0.70019869]
0.6300537667892981
0.19931467456872223


In [7]:
# Fit the Lasso model
lasso.fit(X_train, y_train)

E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
<ipython-input-7-ce4b8b42917a>:2: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(X_train, y_train)
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no reg

Lasso(alpha=0.0, normalize=True)

In [8]:
y_pred = lasso.predict(X_test)

In [9]:
y_pred

array([93.48284674, 86.85909585, 95.73165045, 94.13711657, 78.86915498,
       85.30209449, 90.40056457, 91.28283591, 95.85514765, 88.67630857,
       85.66174867, 85.45132807, 87.78819546, 78.96112145, 30.23583812,
       75.98998944, 93.57801771, 91.90112769, 89.84837655, 81.39490742,
       92.32032723, 91.95771361, 84.54133911, 85.74196827, 88.26513938,
       86.89750788, 85.73498004, 88.24947735, 83.41750447, 91.6230426 ,
       90.55806047, 83.22933853, 71.06328255, 86.88892765, 96.90009821,
       88.08902523, 90.80224754, 90.58131057, 93.97118038, 90.46755221,
       88.35448151, 89.19369884, 89.20121684, 94.65711128, 88.13137529,
       92.22434985, 82.04074932, 86.68418281, 88.4561561 , 95.41886279,
       96.52096269, 96.18116495, 92.88717273, 94.93595066, 85.06995367,
       87.96525558, 73.86654771, 92.46578484, 95.65694869, 92.69493801,
       76.16783778, 92.00187719, 91.61473673, 95.34045726, 92.11347209,
       83.47606552, 76.75064861, 84.68537304, 90.83047329, 94.67

In [10]:
# Create the model score
lasso.score(X_test, y_test)

0.7708194068510523

In [11]:
# dump(lasso, 'C:/Users/Minet/Documents/Repositories/Dev10_Final_Project/Datasets/joblib.model')